### Práctica de Web Scraping

Se va a considerar una de las páginas vista en la teoría que contiene fotos de trenes:https://www.vialibre-ffe.com/multi_foto.asp?cs=mult. En esta página aparece un menú con varios grupos de fotos de trenes. A su vez, cada grupo contiene un conjunto de subgrupos de fotos. Por ejemplo, el grupo __Renfe operadora__ se encuentra en la página https://www.vialibre-ffe.com/multi_ind_fotos.asp?cat=mu01.

In [ ]:
from IPython.display import Image
Image(filename='Captura1.png')

Dentro de este grupo por ejemplo se encuentra el subgrupo __Serie120 en Irún-Hendaya__ en la página https://www.vialibre-ffe.com/multi_galeria.asp?gal=524. 

In [ ]:
from IPython.display import Image
Image(filename='Captura2.png')

En dicho subgrupo se encuentran las imágenes de dicho subgrupo. Por ejemplo, a continuación, se muestra una de las imágenes del subgrupo __Serie120 en Irún-Hendaya__

In [ ]:
from IPython.display import Image
Image(filename='Captura1.png')
#for i in range(1,4):
#    Image(filename='Captura'+str(i)+'.png')

Las imágenes contenidas en dicha página se pueden encontrar en las etiquetas __&lt;li> &lt;img ...__ que son hijas de la etiqueta __&lt;ul class="pgwSlideshow">__. Por ejemplo en la página de ejemplo, la primera imagen de dicha página es:

   __&lt;li>&lt;img src="multimedia/galerias/IRUN120/2Alvia_120.jpg" alt="" data-description="">&lt;/li>__
   
A continuación, se puede ver el contenido html de esa página.

In [ ]:
import requests
url="https://www.vialibre-ffe.com/multi_galeria.asp?gal=524"
r = requests.get(url)
html = r.text
print(html)

Se pide:

 1. Crear un programa en __Python__ que muestre un menú con los grupos de fotos, y que pregunte al usuario que grupo de fotos quiere visitar. Se puede asociar a cada grupo un número, y solicitar al usuario que introduzca por teclado el número del grupo. A continuación, se mostrará los subgrupos de ese grupo, y se le preguntará nuevamente al usuario que subgrupo de fotos quiere procesar. Igual que antes, se puede asociar a cada subgrupo un número, y que introduzca por teclado el número del subgrupo. Como resultado se creará un directorio en el disco local para el subgrupo elegido y en el directorio se bajarán las imágenes de los trenes. Se imprimirá por pantalla las urls de las imágenes que se están bajando. Toda la información utilizada en el programa, debe ser extraida de las páginas consideradas usando BeautifulSoup[6 puntos]
 
 2. Crear un minibuscador en __Python__ que pregunte al usuario un conjunto de palabras clave, y  recorra el sitio web buscando todos los subgrupos en cuyo título aparezca alguna de las palabras claves. Como resultado debe mostrar los subgrupos encontrados, listando el nombre y la url de cada subgrupo [4 puntos]

# Normas de entrega

* Fecha tope de entrega: 18/11/2017
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre WebScraping_GrupoX donde X será el número de grupo correspondiente.


## Ejercicio 1
Crear un programa en Python que muestre un menú con los grupos de fotos, y que pregunte al usuario que grupo de fotos quiere visitar. Se puede asociar a cada grupo un número, y solicitar al usuario que introduzca por teclado el número del grupo. A continuación, se mostrará los subgrupos de ese grupo, y se le preguntará nuevamente al usuario que subgrupo de fotos quiere procesar. Igual que antes, se puede asociar a cada subgrupo un número, y que introduzca por teclado el número del subgrupo. Como resultado se creará un directorio en el disco local para el subgrupo elegido y en el directorio se bajarán las imágenes de los trenes. Se imprimirá por pantalla las urls de las imágenes que se están bajando. Toda la información utilizada en el programa, debe ser extraida de las páginas consideradas usando BeautifulSoup[6 puntos]

In [ ]:
from bs4 import BeautifulSoup
from termcolor import colored
import urllib
import os
import urllib.request

def dig_gallery(url):
    """Obtiene la informacion de las imagenes y las almacena en un diccionario.
    
    Parámetros de entrada:
    url --- direccion de la que obtener la informacion
    
    Retorno:
    gallery --- diccionario con los datos de las imagenes (titulo, url)
    """
    
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    index = 0
    gallery = {}
    for label in soup("a",{'class': 'tit_art_dest'}):
        tag = label.get("href",None)
        name = label.contents[0]
        gallery[index] = (name.strip(), tag.strip())
        index = index + 1
    return gallery

def show_gallery(gallery):
    """Muestra el nombre de las galerias disponibles.
    
    Parámetros de entrada:
    gallery --- diccionario con la informacion de las galerias
    """
    
    for x in gallery:
        print (str(x) + " - " + gallery[x][0])

def dig_images(url):
    """Obtiene las url de las imagenes y las almacena en un diccionario.
    
    Parámetros de entrada:
    url --- direccion de la que obtener la informacion
    
    Retorno:
    images --- diccionario con las url de las imagenes
    """
    
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    images = None
    for label in soup("ul",{'class': 'pgwSlideshow'}):
        images = label.find_all('img')
    return images

def ask_option(gallery):
    """Solicita al usuario la galeria deseada y devuelve su nombre y url.
    
    Parámetros de entrada:
    gallery --- diciconario con la informacion de las galerias (titulo, url)
    
    Retorno:
    gallery[i][0] --- titulo de la galeria i-esima
    gallery[i][1] --- url de la galeria i-esima
    """
    
    try:
        option = int(input('\nSelecciona una galeria: '))
        if option < 0 or option > len(gallery): raise ValueError
        print("\nSe ha seleccionado: ", gallery[option][0], '\n')
    except ValueError:
        print(colored('Error con el argumento proporcionado', 'red'))
        raise ValueError
    return gallery[option][0], gallery[option][1]

def download_image(url):
    """Descarga una imagen dada una url
    
    Parámetros de entrada:
    url --- url de la que descargar la imagen
    """
    
    img = urllib.request.urlopen(url)
    words = url.split('/')
    img_name = words[-1]
    with open(img_name,"wb") as img_file:
        while True:
            info = img.read(100000)
            if len(info) < 1:
                break
            img_file.write(info)

def download_gallery(images): 
    """Descarga una galeria de imagenes
    
    Parámetros de entrada:
    images --- diccionario con las url de las imagenes
    """
    
    for img in images:
        img_src = img.get('src').strip()
        url_img = "https://www.vialibre-ffe.com/" + img_src
        print(url_img)
        download_image(url_img)
    os.chdir("..") # salimos del directorio que hemos llenado para futuras ejecuciones

def create_dir(file):
    """Crea un directorio en el que descargar las imagenes
    
    Parámetros de entrada:
    file --- nombre del directorio a crear
    """
    
    try:
        os.makedirs(file)
        print('Se ha creado el directorio', file)
    except FileExistsError:
        print(colored('El directorio ' + file + ' ya existe', 'red'))
    os.chdir(file) # accedemos al directorio nuevo para descargar las imagenes

# METODO SOLICITADO EN EL ENUNCIADO
def extract_images():
    """Descarga una galeria de imagenes en un directorio
    """
    
    try:
        url = "https://www.vialibre-ffe.com/multi_foto.asp?cs=mult"
        """Obtenemos la galeria principal, la mostramos y le pedimos al usuario que selecccione una. De
        ahi obtenemos el nombre de la galeria seleccionada y su url
        """
        gallery = dig_gallery(url)
        show_gallery(gallery)
        name, new_url = ask_option(gallery)
        
        """Obtenemos la sub galeria indicada por el usuario, la mostramos y le pedimos de nuevo que
        seleccione una. Obtenemos de nuevo el nombre y la url
        """
        url = "https://www.vialibre-ffe.com/" + new_url
        sub_gallery = dig_gallery(url)
        show_gallery(sub_gallery)
        name, new_url = ask_option(sub_gallery)
        
        """Creamos un directorio en el disco local con el nombre de la sub galeria. Obtenemos las url
        de las imagenes de dicha galeria y descargamos en ese directorio dichas imagenes
        """
        url = "https://www.vialibre-ffe.com/" + new_url       
        create_dir(name)
        images = dig_images(url)
        download_gallery(images)
        print(colored('Fin del ejercicio 1', 'green'))
    except ValueError:
        print(colored('Fin de la ejecucion', 'red'))


## Ejercicio 2
Crear un minibuscador en __Python__ que pregunte al usuario un conjunto de palabras clave, y  recorra el sitio web buscando todos los subgrupos en cuyo título aparezca alguna de las palabras claves. Como resultado debe mostrar los subgrupos encontrados, listando el nombre y la url de cada subgrupo [4 puntos]

In [ ]:
def match_words(query, title):
    """Busca coincidencias entre la informacion del usuario y la informacion de las galerias
    
    Parámetros de entrada:
    query --- lista de palabras proporcionadas por el usuario
    title --- cadena de texto con titulos sobre los que hacer busquedas
    
    Retorno:
    True --- hay coincidencias
    False --- no hay coincidencias
    """
    
    for word in query:
        if word in title.lower().split():
            return True
    return False

def parse_title(gallery, query):
    """Muestra la informacion de la galeria si hay coincidencias con la info del usuario
    
    Parámetros de entrada:
    gallery --- diciconario con la informacion de las galerias (titulo, url)
    query --- lista con las palabras del usuario
    
    Retorno:
    url --- url a una pagina donde hay coincidencia
    """
    
    title, section = gallery[0], gallery[1]
    url = "https://www.vialibre-ffe.com/" + section
    if match_words(query, title): print (title + ": " + url)
    return url

# METODO SOLICITADO EN EL ENUNCIADO
def search_engine():
    """Minibuscador que imprime por pantalla el titulo y la url si contienen la info proporcionada
    """
    
    words = input("\n\n\nIntroduce las palabras a buscar: ")
    query = words.lower().split()
    
    url = "https://www.vialibre-ffe.com/multi_foto.asp?cs=mult"
    gallery = dig_gallery(url)

    for elem in gallery:
        url = parse_title(gallery[elem], query)
        sub_gallery = dig_gallery(url)
        for sub_elem in sub_gallery:
            url = parse_title(sub_gallery[sub_elem], query)

## Ejecucion

In [ ]:
extract_images()
search_engine()